In [1]:
from sklearn.model_selection import train_test_split
import torch
import pandas as pd

In [2]:
#split dataframe train test
def split_train_test_stratified(test_size):
    df = pd.read_csv('processed_shape_data.csv')
    df['num_label'] = df.positive*0 + df.neutral*1 + df.negative*2
    X = df[['sessionID','path','labels','positive','neutral','negative']]
    Y = df["num_label"]
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, stratify=Y,random_state=0)
    return x_train, x_test, y_train, y_test

In [3]:
#create label files.pt to be output
def create_train_test_label_file(x_train, x_test):
    label_train = x_train[['positive','neutral','negative']]
    tensor_label_train = torch.tensor(label_train.values)
    torch.save(tensor_label_train,'label_train.pt')
    label_test = x_test[['positive','neutral','negative']]
    tensor_label_test = torch.tensor(label_test.values)
    torch.save(tensor_label_test,'label_test.pt')

In [22]:
#create data files.pt to be input
def create_train_test_data_file(x_train, x_test):
    tmp_file = torch.empty((len(x_train),128,1500))
    count = 0
    for file in x_train['path']:
        tmp_file[count] = torch.load(file)
        count+=1
    torch.save(tmp_file,'data_train.pt')
    tmp_file = torch.empty((len(x_test),128,1500))
    count = 0
    for file in x_test['path']:
        tmp_file[count] = torch.load(file)
        count+=1
    torch.save(tmp_file,'data_test.pt')

In [27]:
def main(test_size):
    x_train, x_test, y_train, y_test = split_train_test_stratified(test_size)
    create_train_test_label_file(x_train,x_test)
    create_train_test_data_file(x_train,x_test)

In [28]:
main(test_size = 0.2)

In [33]:
def print_shape():
    list_ = ['data_train','data_test','label_train','label_test']
    for name in list_:
        name_file = name+'.pt'
        tensor = torch.load(name_file)
        print(name,' has shape: ',tensor.shape)

In [34]:
print_shape()

data_train  has shape:  torch.Size([6023, 128, 1500])
data_test  has shape:  torch.Size([1506, 128, 1500])
label_train  has shape:  torch.Size([6023, 3])
label_test  has shape:  torch.Size([1506, 3])
